In [13]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score,
                             mean_absolute_percentage_error, explained_variance_score,
                             accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report,
                             roc_auc_score, roc_curve)
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
warnings.filterwarnings('ignore')

In [10]:
# ============================================
# ตรวจสอบว่ามี GPU พร้อมใช้งานหรือไม่
# ============================================
print(f"\n{'='*60}")
print("🖥️  CHECKING GPU AVAILABILITY")
print(f"{'='*60}")

# ตรวจสอบ CUDA/GPU สำหรับ XGBoost
gpu_available = False
try:
    # ลองสร้าง XGBoost model ด้วย GPU
    test_model = xgb.XGBRegressor(tree_method='hist', gpu_id=0)
    gpu_available = True
    print("✅ GPU is AVAILABLE and will be used for training")
    print(f"   Using: CUDA/GPU acceleration")
    tree_method = 'hist'  # ใช้ GPU
except:
    print("⚠️  GPU is NOT available, using CPU instead")
    print(f"   Using: CPU (hist method)")
    tree_method = 'hist'  # ใช้ CPU แบบเร็ว

# แสดงข้อมูล XGBoost version
print(f"   XGBoost version: {xgb.__version__}")

# ถ้าต้องการดูรายละเอียด GPU เพิ่มเติม (ต้องมี nvidia-smi)
try:
    import subprocess
    result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], 
                          capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print(f"   GPU Info: {result.stdout.strip()}")
except:
    pass

print(f"{'='*60}\n")



🖥️  CHECKING GPU AVAILABILITY
✅ GPU is AVAILABLE and will be used for training
   Using: CUDA/GPU acceleration
   XGBoost version: 3.1.1
   GPU Info: NVIDIA GeForce RTX 3060 Laptop GPU, 6144 MiB



In [4]:

# ============================================
# โหลดและเตรียมข้อมูล
# ============================================
print(f"{'='*60}")
print("📂 LOADING AND PREPARING DATA")
print(f"{'='*60}\n")

# อ่านไฟล์ข้อมูล
df = pd.read_csv('../dataset/accident_count4col.csv')
df['adate'] = pd.to_datetime(df['adate'])
df['month'] = df['adate'].dt.month

# กรองเฉพาะเดือน ธันวาคม (12) และ มกราคม (1)
df = df[df['month'].isin([12, 1])].copy()
print(f"✓ Filtered data: December and January only")

# สร้าง group_id จากการรวม rcode + aampur_clean + aplace_clean
df['group_id'] = (df['rcode'].astype(str) + '_' + 
                  df['aampur_clean'].astype(str) + '_' + 
                  df['aplace_clean'].astype(str))

print(f"✓ Total rows: {len(df):,}")
print(f"✓ Unique groups: {df['group_id'].nunique()}")
print(f"✓ Date range: {df['adate'].min()} to {df['adate'].max()}")


📂 LOADING AND PREPARING DATA

✓ Filtered data: December and January only
✓ Total rows: 172,935
✓ Unique groups: 952
✓ Date range: 2018-01-01 00:00:00 to 2025-01-31 00:00:00


In [5]:

# ============================================
# Feature Engineering (สร้างฟีเจอร์เพิ่มเติม)
# ============================================
print(f"\n{'='*60}")
print("🔧 FEATURE ENGINEERING")
print(f"{'='*60}\n")

# ฟีเจอร์พื้นฐานจากวันที่
df['year'] = df['adate'].dt.year
df['day_of_month'] = df['adate'].dt.day
df['day_of_week'] = df['adate'].dt.dayofweek  # 0=จันทร์, 6=อาทิตย์
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)  # เสาร์-อาทิตย์
df['is_december'] = (df['month'] == 12).astype(int)  # เดือนธันวาคมหรือไม่

print(f"✓ Basic time features created")

# ฟีเจอร์จากข้อมูลกลุ่ม (Group-level features)
# คำนวณค่าเฉลี่ยของแต่ละกลุ่มสถานที่
group_means = df.groupby('group_id')['acc_cases'].mean()
df['group_mean'] = df['group_id'].map(group_means)

# คำนวณค่าเบี่ยงเบนมาตรฐานของแต่ละกลุ่ม
group_std = df.groupby('group_id')['acc_cases'].std().fillna(0)
df['group_std'] = df['group_id'].map(group_std)

print(f"✓ Group statistics features created")

# Lag Features (ข้อมูลย้อนหลัง)
# เรียงข้อมูลตามกลุ่มและวันที่
df = df.sort_values(['group_id', 'adate'])

# ข้อมูลย้อนหลัง 1 ปี (365 วัน)
df['lag_1year'] = df.groupby('group_id')['acc_cases'].shift(365)

# ข้อมูลย้อนหลัง 2 ปี (730 วัน)
df['lag_2year'] = df.groupby('group_id')['acc_cases'].shift(730)

print(f"✓ Lag features created (1-year, 2-year)")

# Rolling Features (ค่าเฉลี่ยเคลื่อนที่)
# ค่าเฉลี่ยของ 7 วันที่ผ่านมา
df['rolling_mean_7d'] = df.groupby('group_id')['acc_cases'].rolling(7, min_periods=1).mean().reset_index(0, drop=True)

# ค่าเฉลี่ยของ 14 วันที่ผ่านมา
df['rolling_mean_14d'] = df.groupby('group_id')['acc_cases'].rolling(14, min_periods=1).mean().reset_index(0, drop=True)

print(f"✓ Rolling window features created (7-day, 14-day)")

# เติมค่า NaN ด้วย 0
df = df.fillna(0)

print(f"✓ Missing values filled")
print(f"✓ Total features created: {len(df.columns)}")


🔧 FEATURE ENGINEERING

✓ Basic time features created
✓ Group statistics features created
✓ Lag features created (1-year, 2-year)
✓ Rolling window features created (7-day, 14-day)
✓ Missing values filled
✓ Total features created: 18


In [6]:


# ============================================
# แบ่งข้อมูล Train/Test
# ============================================
print(f"\n{'='*60}")
print("✂️  SPLITTING DATA")
print(f"{'='*60}\n")

# Train: ข้อมูลก่อน 2024-01-31
train = df[df['adate'] <= '2024-01-31'].copy()

# Test: ข้อมูลระหว่าง 2024-12-01 ถึง 2025-01-31
test = df[(df['adate'] >= '2024-12-01') & (df['adate'] <= '2025-01-31')].copy()

print(f"✓ Train set: {len(train):,} rows ({train['adate'].min()} to {train['adate'].max()})")
print(f"✓ Test set:  {len(test):,} rows ({test['adate'].min()} to {test['adate'].max()})")

# กำหนด Feature columns (คอลัมน์ที่จะใช้ในการทำนาย)
feature_cols = [
    'month', 'day_of_month', 'day_of_week', 'is_weekend', 
    'is_december', 'year', 'rcode', 'aampur_clean', 'aplace_clean',
    'group_mean', 'group_std', 'lag_1year', 'lag_2year', 
    'rolling_mean_7d', 'rolling_mean_14d'
]

# แยก Features (X) และ Target (y)
X_train = train[feature_cols]
y_train = train['acc_cases']
X_test = test[feature_cols]
y_test = test['acc_cases']

print(f"✓ Features: {len(feature_cols)} columns")
print(f"✓ Target: acc_cases")




✂️  SPLITTING DATA

✓ Train set: 138,438 rows (2018-01-01 00:00:00 to 2024-01-31 00:00:00)
✓ Test set:  34,497 rows (2024-12-01 00:00:00 to 2025-01-31 00:00:00)
✓ Features: 15 columns
✓ Target: acc_cases


In [11]:
# ============================================
# สร้างและเทรนโมเดล XGBoost
# ============================================
print(f"\n{'='*60}")
print("🤖 TRAINING XGBOOST MODEL")
print(f"{'='*60}\n")

# สร้างโมเดล XGBoost พร้อมพารามิเตอร์ที่ปรับแต่ง
model = xgb.XGBRegressor(
    n_estimators=200,          # จำนวน trees (ยิ่งมากยิ่งแม่นแต่ช้า)
    learning_rate=0.05,        # อัตราการเรียนรู้ (ต่ำ = ช้าแต่แม่นกว่า)
    max_depth=6,               # ความลึกสูงสุดของ tree
    min_child_weight=1,        # น้ำหนักขั้นต่ำของ leaf node
    subsample=0.8,             # สุ่มตัวอย่างข้อมูล 80% ในแต่ละรอบ
    colsample_bytree=0.8,      # สุ่มตัวอย่างฟีเจอร์ 80% ในแต่ละ tree
    tree_method=tree_method,   # ใช้ GPU หรือ CPU ตามที่ตรวจสอบไว้
    device='cuda',              # หรือ 'cpu'
    random_state=42,           # กำหนด seed เพื่อให้ผลลัพธ์ซ้ำได้
    n_jobs=-1                  # ใช้ CPU ทุก core (สำหรับ CPU mode)
)

print(f"Model Configuration:")
print(f"  • Tree method: {tree_method} {'(GPU ⚡)' if gpu_available else '(CPU)'}")
print(f"  • Number of trees: 200")
print(f"  • Learning rate: 0.05")
print(f"  • Max depth: 6")
print(f"\nTraining model... (This may take a while)")

# เทรนโมเดล
import time
start_time = time.time()

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],  # ใช้ test set สำหรับ validation
    verbose=50                     # แสดงผลทุก 50 iterations
)

training_time = time.time() - start_time

print(f"\n✅ Training completed!")
print(f"   Training time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
print(f"   Speed: {len(train)/training_time:.0f} samples/second")


🤖 TRAINING XGBOOST MODEL

Model Configuration:
  • Tree method: hist (GPU ⚡)
  • Number of trees: 200
  • Learning rate: 0.05
  • Max depth: 6

Training model... (This may take a while)
[0]	validation_0-rmse:4.33900
[50]	validation_0-rmse:2.57273
[100]	validation_0-rmse:2.53758
[150]	validation_0-rmse:2.54101
[199]	validation_0-rmse:2.54717

✅ Training completed!
   Training time: 0.90 seconds (0.02 minutes)
   Speed: 152990 samples/second


In [15]:
# ============================================
# 💾 SAVE MODEL (บันทึกโมเดล)
# ============================================
print(f"{'='*60}")
print("💾 SAVING MODEL")
print(f"{'='*60}\n")

# สร้างโฟลเดอร์สำหรับเก็บโมเดล
os.makedirs('models', exist_ok=True)

# บันทึกโมเดล XGBoost (2 วิธี)

# วิธีที่ 1: XGBoost JSON format (แนะนำ - รองรับข้าม platform)
model_path_json = 'models/xgboost_accident_model.json'
model.save_model(model_path_json)
print(f"✓ Saved: {model_path_json}")

# วิธีที่ 2: Python Pickle format (สำหรับใช้ใน Python เท่านั้น)
model_path_pkl = 'models/xgboost_accident_model.pkl'
with open(model_path_pkl, 'wb') as f:
    pickle.dump(model, f)
print(f"✓ Saved: {model_path_pkl}")

# บันทึก feature columns (สำคัญ!)
feature_info = {
    'feature_columns': feature_cols,
    'model_type': 'XGBRegressor',
    'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'device': 'gpu',
    'training_samples': len(train),
    'xgboost_version': xgb.__version__
}

import json
with open('models/model_info.json', 'w') as f:
    json.dump(feature_info, f, indent=2)
print(f"✓ Saved: models/model_info.json")

print(f"\n📂 Model files saved in 'models/' folder\n")

💾 SAVING MODEL

✓ Saved: models/xgboost_accident_model.json
✓ Saved: models/xgboost_accident_model.pkl
✓ Saved: models/model_info.json

📂 Model files saved in 'models/' folder



In [16]:


# ============================================
# COMPREHENSIVE MODEL EVALUATION
# ============================================
print(f"\n{'='*60}")
print("📊 MODEL EVALUATION - COMPREHENSIVE METRICS")
print(f"{'='*60}")

# ทำนายผลลัพธ์
print(f"\nMaking predictions...")
start_pred = time.time()
y_pred = model.predict(X_test)
pred_time = time.time() - start_pred

y_pred = np.maximum(0, y_pred)  # ไม่ให้ค่าติดลบ (จำนวนอุบัติเหตุต้อง >= 0)

print(f"✓ Prediction time: {pred_time:.2f} seconds")
print(f"✓ Speed: {len(test)/pred_time:.0f} predictions/second")




📊 MODEL EVALUATION - COMPREHENSIVE METRICS

Making predictions...
✓ Prediction time: 0.01 seconds
✓ Speed: 2364617 predictions/second


In [ ]:
# ============================================
# REGRESSION METRICS (สำหรับทำนายจำนวนอุบัติเหตุ)
# ============================================
print(f"\n📊 REGRESSION METRICS:")
print(f"{'─'*60}")

# คำนวณ Metrics พื้นฐาน
mae = mean_absolute_error(y_test, y_pred)  # ค่าเฉลี่ยความผิดพลาดสัมบูรณ์
mse = mean_squared_error(y_test, y_pred)   # ค่าเฉลี่ยความผิดพลาดกำลังสอง
rmse = np.sqrt(mse)                        # รากที่สองของ MSE
r2 = r2_score(y_test, y_pred)              # สัมประสิทธิ์การกำหนด (0-1)

# MAPE - ป้องกันการหารด้วย 0
mape = np.mean(np.abs((y_test - y_pred) / np.where(y_test == 0, 1, y_test))) * 100

# Metrics เพิ่มเติม
explained_var = explained_variance_score(y_test, y_pred)  # ความแปรปรวนที่อธิบายได้
median_ae = np.median(np.abs(y_test - y_pred))            # ค่ามัธยฐานของความผิดพลาด
max_error = np.max(np.abs(y_test - y_pred))               # ความผิดพลาดสูงสุด

print(f"  MAE (Mean Absolute Error):          {mae:.4f}")
print(f"    → ค่าเฉลี่ยของความคลาดเคลื่อน")
print(f"  MSE (Mean Squared Error):           {mse:.4f}")
print(f"  RMSE (Root Mean Squared Error):     {rmse:.4f}")
print(f"    → ให้น้ำหนักกับความผิดพลาดมาก")
print(f"  MAPE (Mean Absolute % Error):       {mape:.2f}%")
print(f"    → ความคลาดเคลื่อนเป็นเปอร์เซ็นต์")
print(f"  R² Score:                           {r2:.4f}")
print(f"    → ความสามารถในการอธิบายข้อมูล (ยิ่งใกล้ 1 ยิ่งดี)")
print(f"  Explained Variance:                 {explained_var:.4f}")
print(f"  Median Absolute Error:              {median_ae:.4f}")
print(f"  Max Error:                          {max_error:.4f}")

# ============================================
# PREDICTION DISTRIBUTION ANALYSIS
# ============================================
print(f"\n📈 PREDICTION DISTRIBUTION:")
print(f"{'─'*60}")
print(f"  Actual Total Cases:      {y_test.sum():>10,.0f}")
print(f"  Predicted Total Cases:   {y_pred.sum():>10,.0f}")
print(f"  Difference:              {(y_pred.sum() - y_test.sum()):>10,.0f}")
print(f"  Total Accuracy:          {(1 - abs(y_pred.sum() - y_test.sum()) / y_test.sum()) * 100:>10.2f}%")

print(f"\n  Actual Statistics:")
print(f"    Min:     {y_test.min():.2f}")
print(f"    25%:     {np.percentile(y_test, 25):.2f}")
print(f"    Median:  {np.median(y_test):.2f}")
print(f"    Mean:    {y_test.mean():.2f}")
print(f"    75%:     {np.percentile(y_test, 75):.2f}")
print(f"    Max:     {y_test.max():.2f}")

print(f"\n  Predicted Statistics:")
print(f"    Min:     {y_pred.min():.2f}")
print(f"    25%:     {np.percentile(y_pred, 25):.2f}")
print(f"    Median:  {np.median(y_pred):.2f}")
print(f"    Mean:    {y_pred.mean():.2f}")
print(f"    75%:     {np.percentile(y_pred, 75):.2f}")
print(f"    Max:     {y_pred.max():.2f}")

# ============================================
# PER-GROUP EVALUATION (ประเมินแต่ละกลุ่มสถานที่)
# ============================================
print(f"\n🏘️  PER-GROUP EVALUATION:")
print(f"{'─'*60}")

test_copy = test.copy()
test_copy['predicted'] = y_pred
group_metrics = []

# วนลูปคำนวณ metrics สำหรับแต่ละกลุ่ม
for group_id in test_copy['group_id'].unique():
    group_test = test_copy[test_copy['group_id'] == group_id]
    y_true = group_test['acc_cases']
    y_pred_group = group_test['predicted']
    
    # คำนวณ metrics
    group_mae = mean_absolute_error(y_true, y_pred_group)
    group_rmse = np.sqrt(mean_squared_error(y_true, y_pred_group))
    group_r2 = r2_score(y_true, y_pred_group) if len(y_true) > 1 else 0
    
    group_metrics.append({
        'group_id': group_id,
        'mae': group_mae,
        'rmse': group_rmse,
        'r2': group_r2,
        'n_samples': len(group_test),
        'actual_sum': y_true.sum(),
        'predicted_sum': y_pred_group.sum(),
        'accuracy': (1 - abs(y_pred_group.sum() - y_true.sum()) / (y_true.sum() + 1)) * 100
    })

metrics_df = pd.DataFrame(group_metrics)

print(f"  Total Groups Evaluated:  {len(metrics_df)}")
print(f"  Average MAE:             {metrics_df['mae'].mean():.4f}")
print(f"  Average RMSE:            {metrics_df['rmse'].mean():.4f}")
print(f"  Average R²:              {metrics_df['r2'].mean():.4f}")
print(f"  Best MAE:                {metrics_df['mae'].min():.4f}")
print(f"  Worst MAE:               {metrics_df['mae'].max():.4f}")

# บันทึกผลลัพธ์ละเอียด
metrics_df.to_csv('evaluation_metrics_per_group.csv', index=False)
print(f"\n  ✓ Saved to: evaluation_metrics_per_group.csv")

# แสดง 10 กลุ่มที่ทำนายได้แย่ที่สุด
print(f"\n❌ TOP 10 WORST PERFORMING GROUPS:")
print(f"{'─'*60}")
worst = metrics_df.nlargest(10, 'mae')[['group_id', 'mae', 'rmse', 'r2', 'actual_sum', 'predicted_sum']]
print(worst.to_string(index=False))

# แสดง 10 กลุ่มที่ทำนายได้ดีที่สุด
print(f"\n✅ TOP 10 BEST PERFORMING GROUPS:")
print(f"{'─'*60}")
best = metrics_df.nsmallest(10, 'mae')[['group_id', 'mae', 'rmse', 'r2', 'actual_sum', 'predicted_sum']]
print(best.to_string(index=False))

# ============================================
# FEATURE IMPORTANCE (ความสำคัญของแต่ละฟีเจอร์)
# ============================================
print(f"\n🔍 FEATURE IMPORTANCE:")
print(f"{'─'*60}")
print(f"  (แสดงว่าฟีเจอร์ไหนมีผลต่อการทำนายมากที่สุด)")

importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(importance.to_string(index=False))
importance.to_csv('feature_importance.csv', index=False)
print(f"\n  ✓ Saved to: feature_importance.csv")

# ============================================
# VISUALIZATION (สร้างกราฟแสดงผล)
# ============================================
print(f"\n📊 GENERATING VISUALIZATIONS:")
print(f"{'─'*60}")

# 1. Actual vs Predicted Scatter Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5, s=10)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Cases', fontsize=12) # (จำนวนจริง)
plt.ylabel('Predicted Cases', fontsize=12) # (จำนวนที่ทำนาย)
plt.title('Actual vs Predicted Cases', fontsize=14) #\n(จุดที่อยู่ใกล้เส้นแดง = ทำนายแม่นยำ)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('evaluation_scatter_plot.png', dpi=300, bbox_inches='tight')
print(f"  ✓ Saved: evaluation_scatter_plot.png")
plt.close()

# 2. Residuals Plot (กราฟแสดงความคลาดเคลื่อน)
plt.figure(figsize=(10, 6))
residuals = y_test - y_pred
plt.scatter(y_pred, residuals, alpha=0.5, s=10)
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Predicted Cases', fontsize=12) #(จำนวนที่ทำนาย)
plt.ylabel('Residuals', fontsize=12) #  (ความคลาดเคลื่อน)
plt.title('Residual Plot)', fontsize=14) #\n(จุดควรกระจายรอบเส้น 0 = โมเดลดี
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('evaluation_residuals.png', dpi=300, bbox_inches='tight')
print(f"  ✓ Saved: evaluation_residuals.png")
plt.close()

# 3. Feature Importance Plot
plt.figure(figsize=(10, 8))
top_n = 15
importance_top = importance.head(top_n)
colors = plt.cm.viridis(np.linspace(0, 1, len(importance_top)))
plt.barh(range(len(importance_top)), importance_top['importance'], color=colors)
plt.yticks(range(len(importance_top)), importance_top['feature'])
plt.xlabel('Importance Score', fontsize=12)
plt.title(f'Top {top_n} Feature Importance\n(ฟีเจอร์ที่สำคัญที่สุด)', fontsize=14)
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('evaluation_feature_importance.png', dpi=300, bbox_inches='tight')
print(f"  ✓ Saved: evaluation_feature_importance.png")
plt.close()




📊 REGRESSION METRICS:
────────────────────────────────────────────────────────────
  MAE (Mean Absolute Error):          1.7143
    → ค่าเฉลี่ยของความคลาดเคลื่อน
  MSE (Mean Squared Error):           6.4881
  RMSE (Root Mean Squared Error):     2.5472
    → ให้น้ำหนักกับความผิดพลาดมาก
  MAPE (Mean Absolute % Error):       63.86%
    → ความคลาดเคลื่อนเป็นเปอร์เซ็นต์
  R² Score:                           0.6736
    → ความสามารถในการอธิบายข้อมูล (ยิ่งใกล้ 1 ยิ่งดี)
  Explained Variance:                 0.6736
  Median Absolute Error:              1.1748
  Max Error:                          39.9129

📈 PREDICTION DISTRIBUTION:
────────────────────────────────────────────────────────────
  Actual Total Cases:         144,302
  Predicted Total Cases:      143,679
  Difference:                    -623
  Total Accuracy:               99.57%

  Actual Statistics:
    Min:     1.00
    25%:     1.00
    Median:  3.00
    Mean:    4.18
    75%:     5.00
    Max:     70.00

  Predicted Statistics

: 

In [20]:
# ============================================
# MODEL EVALUATION (แบบง่าย - KISS)
# ============================================
print(f"{'='*60}")
print("📊 MODEL EVALUATION")
print(f"{'='*60}\n")

# ทำนาย
y_pred = model.predict(X_test)
y_pred = np.maximum(0, y_pred)  # ไม่ให้ติดลบ

# ============================================
# 1. Regression Metrics (สำหรับ Time-Series Forecast)
# ============================================
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / np.where(y_test == 0, 1, y_test))) * 100

print("📈 TIME-SERIES FORECAST METRICS:")
print(f"{'─'*60}")
print(f"  MAE (Mean Absolute Error):     {mae:.4f}")
print(f"    → เฉลี่ยผิดพลาด ±{mae:.2f} cases/day")
print(f"\n  RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"    → เน้นลงโทษความผิดพลาดใหญ่")
print(f"\n  R² Score:                       {r2:.4f}")
print(f"    → อธิบายข้อมูลได้ {r2*100:.1f}%")
print(f"\n  MAPE (Mean Absolute % Error):   {mape:.2f}%")
print(f"    → เฉลี่ยผิดพลาด {mape:.1f}%")

# ตีความผล
print(f"\n💡 INTERPRETATION:")
if r2 > 0.8:
    print(f"  ⭐⭐⭐ Excellent prediction (R² > 0.8)")
elif r2 > 0.6:
    print(f"  ⭐⭐ Good prediction (R² > 0.6)")
elif r2 > 0.4:
    print(f"  ⭐ Moderate prediction (R² > 0.4)")
else:
    print(f"  ⚠️  Poor prediction (R² < 0.4)")

# ============================================
# 2. Actual vs Predicted Comparison
# ============================================
print(f"\n{'─'*60}")
print(f"📊 ACTUAL vs PREDICTED TOTALS:")
print(f"{'─'*60}")
print(f"  Actual Total:      {y_test.sum():>10,.0f} cases")
print(f"  Predicted Total:   {y_pred.sum():>10,.0f} cases")
print(f"  Difference:        {(y_pred.sum() - y_test.sum()):>10,.0f} cases")
print(f"  Accuracy:          {(1 - abs(y_pred.sum() - y_test.sum()) / y_test.sum()) * 100:>10.1f}%")

# ============================================
# 3. Per-Group Evaluation (กลุ่มไหนทำนายได้ดี/แย่)
# ============================================
print(f"\n{'─'*60}")
print(f"🏘️  PER-GROUP PERFORMANCE:")
print(f"{'─'*60}")

test_eval = test.copy()
test_eval['predicted'] = y_pred
test_eval['error'] = np.abs(test_eval['acc_cases'] - test_eval['predicted'])

group_metrics = test_eval.groupby('group_id').agg({
    'acc_cases': ['sum', 'mean'],
    'predicted': ['sum', 'mean'],
    'error': 'mean'
}).round(2)

group_metrics.columns = ['actual_sum', 'actual_mean', 'pred_sum', 'pred_mean', 'mae']
group_metrics = group_metrics.reset_index()

print(f"  Total groups evaluated: {len(group_metrics)}")
print(f"  Average MAE per group:  {group_metrics['mae'].mean():.2f}")
print(f"  Best MAE:               {group_metrics['mae'].min():.2f}")
print(f"  Worst MAE:              {group_metrics['mae'].max():.2f}")

# Top 5 worst groups
print(f"\n  ⚠️  TOP 5 WORST PERFORMING GROUPS:")
worst = group_metrics.nlargest(5, 'mae')[['group_id', 'actual_sum', 'pred_sum', 'mae']]
for idx, row in worst.iterrows():
    print(f"     {row['group_id']}: MAE={row['mae']:.2f}, Actual={row['actual_sum']:.0f}, Pred={row['pred_sum']:.0f}")

# Top 5 best groups
print(f"\n  ✅ TOP 5 BEST PERFORMING GROUPS:")
best = group_metrics.nsmallest(5, 'mae')[['group_id', 'actual_sum', 'pred_sum', 'mae']]
for idx, row in best.iterrows():
    print(f"     {row['group_id']}: MAE={row['mae']:.2f}, Actual={row['actual_sum']:.0f}, Pred={row['pred_sum']:.0f}")

# ============================================
# 4. Feature Importance (ฟีเจอร์ไหนสำคัญที่สุด)
# ============================================
print(f"\n{'─'*60}")
print(f"🔍 TOP 10 IMPORTANT FEATURES:")
print(f"{'─'*60}")

importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

for idx, row in importance_df.head(10).iterrows():
    bar_length = int(row['importance'] * 50)
    bar = '█' * bar_length
    print(f"  {row['feature']:20s} {bar} {row['importance']:.4f}")

# บันทึก evaluation results
group_metrics.to_csv('models/evaluation_per_group.csv', index=False)
importance_df.to_csv('models/feature_importance.csv', index=False)
print(f"\n✓ Saved: models/evaluation_per_group.csv")
print(f"✓ Saved: models/feature_importance.csv")

# ============================================
# 5. สรุปผลการประเมิน
# ============================================
print(f"\n{'='*60}")
print(f"📋 EVALUATION SUMMARY")
print(f"{'='*60}")
print(f"  ✓ Model Type:      XGBoost Regressor")
print(f"  ✓ Training Device: GPU")
print(f"  ✓ Training Time:   {training_time:.2f} seconds")
print(f"  ✓ MAE:             {mae:.4f}")
print(f"  ✓ RMSE:            {rmse:.4f}")
print(f"  ✓ R²:              {r2:.4f}")
print(f"  ✓ MAPE:            {mape:.2f}%")
print(f"  ✓ Total Accuracy:  {(1 - abs(y_pred.sum() - y_test.sum()) / y_test.sum()) * 100:.1f}%")
print(f"{'='*60}\n")



📊 MODEL EVALUATION

📈 TIME-SERIES FORECAST METRICS:
────────────────────────────────────────────────────────────
  MAE (Mean Absolute Error):     1.7143
    → เฉลี่ยผิดพลาด ±1.71 cases/day

  RMSE (Root Mean Squared Error): 2.5472
    → เน้นลงโทษความผิดพลาดใหญ่

  R² Score:                       0.6736
    → อธิบายข้อมูลได้ 67.4%

  MAPE (Mean Absolute % Error):   63.86%
    → เฉลี่ยผิดพลาด 63.9%

💡 INTERPRETATION:
  ⭐⭐ Good prediction (R² > 0.6)

────────────────────────────────────────────────────────────
📊 ACTUAL vs PREDICTED TOTALS:
────────────────────────────────────────────────────────────
  Actual Total:         144,302 cases
  Predicted Total:      143,679 cases
  Difference:              -623 cases
  Accuracy:                99.6%

────────────────────────────────────────────────────────────
🏘️  PER-GROUP PERFORMANCE:
────────────────────────────────────────────────────────────
  Total groups evaluated: 924
  Average MAE per group:  1.45
  Best MAE:               0.01
  Worst

In [29]:
# ============================================
# FORECAST FUTURE (2025-12 to 2026-01)
# ============================================
print(f"{'='*60}")
print("🔮 GENERATING FUTURE FORECAST")
print(f"{'='*60}\n")

future_dates = pd.date_range('2025-12-01', '2026-01-31', freq='D')
all_groups = df['group_id'].unique()

print(f"Forecasting:")
print(f"  • Period: {future_dates[0]} to {future_dates[-1]}")
print(f"  • Days: {len(future_dates)}")
print(f"  • Groups: {len(all_groups)}")
print(f"  • Total predictions: {len(future_dates) * len(all_groups):,}\n")

feature_cols = [
    'month', 'day_of_month', 'day_of_week', 'is_weekend', 
    'is_december', 'year', 'rcode', 'aampur_clean', 'aplace_clean',
    'group_mean', 'group_std', 'lag_1year', 'lag_2year', 
    'rolling_mean_7d', 'rolling_mean_14d'
]

forecasts = []
for idx, group in enumerate(all_groups, 1):
    if idx % 100 == 0:
        print(f"  Processing [{idx}/{len(all_groups)}]...")
    
    group_data = df[df['group_id'] == group].iloc[-1]
    
    for date in future_dates:
        forecast_row = {
            'adate': date,
            'group_id': group,
            'month': date.month,
            'day_of_month': date.day,
            'day_of_week': date.dayofweek,
            'is_weekend': 1 if date.dayofweek >= 5 else 0,
            'is_december': 1 if date.month == 12 else 0,
            'year': 2026 if date.month == 1 else 2025,
            'rcode': group_data['rcode'],
            'aampur_clean': group_data['aampur_clean'],
            'aplace_clean': group_data['aplace_clean'],
            'group_mean': group_data['group_mean'],
            'group_std': group_data['group_std'],
            'lag_1year': 0,
            'lag_2year': 0,
            'rolling_mean_7d': group_data['group_mean'],
            'rolling_mean_14d': group_data['group_mean']
        }
        forecasts.append(forecast_row)

print(f"  Processing [{len(all_groups)}/{len(all_groups)}]...")

# Predict
forecast_df = pd.DataFrame(forecasts)
X_forecast = forecast_df[feature_cols]

forecast_df['predicted_cases'] = model.predict(X_forecast)
forecast_df['predicted_cases'] = forecast_df['predicted_cases'].clip(lower=0).round()

# Save
output = forecast_df[['adate', 'rcode', 'aampur_clean', 'aplace_clean', 'predicted_cases']]
output.to_csv('forecast_2025_2026.csv', index=False)

print(f"\n{'='*60}")
print("✅ FORECAST COMPLETE")
print(f"{'='*60}")
print(f"  📊 Total predictions:  {len(output):,} rows")
print(f"  📊 Total cases:        {output['predicted_cases'].sum():,.0f}")
print(f"  📊 Avg per day:        {output.groupby('adate')['predicted_cases'].sum().mean():,.1f}")
print(f"  📁 Saved to:           forecast_2025_2026.csv")
print(f"{'='*60}\n")



🔮 GENERATING FUTURE FORECAST

Forecasting:
  • Period: 2025-12-01 00:00:00 to 2026-01-31 00:00:00
  • Days: 62
  • Groups: 952
  • Total predictions: 59,024

  Processing [100/952]...
  Processing [200/952]...
  Processing [300/952]...
  Processing [400/952]...
  Processing [500/952]...
  Processing [600/952]...
  Processing [700/952]...
  Processing [800/952]...
  Processing [900/952]...
  Processing [952/952]...

✅ FORECAST COMPLETE
  📊 Total predictions:  59,024 rows
  📊 Total cases:        169,950
  📊 Avg per day:        2,741.1
  📁 Saved to:           forecast_2025_2026.csv



# ============================================
# วิธีโหลดโมเดลกลับมาใช้
# ============================================
print(f"{'='*60}")
print("📖 HOW TO LOAD MODEL")
print(f"{'='*60}")
print("""
# วิธีที่ 1: โหลดจาก JSON (แนะนำ)
loaded_model = xgb.XGBRegressor()
loaded_model.load_model('models/xgboost_accident_model.json')

# วิธีที่ 2: โหลดจาก Pickle
import pickle
with open('models/xgboost_accident_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# ใช้ทำนาย
predictions = loaded_model.predict(X_new)
""")
print(f"{'='*60}\n")

print("🎉 ALL DONE!\n")
